In [12]:
from datetime import datetime
import pandas as pd
from pyspark.sql.functions import col, sum

In [13]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [33]:
csv_path = '/Users/bohdanvelychkovych/Downloads/archive/yellow_tripdata_2016-03.csv'

In [34]:
timestamp_start = datetime.now()
df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.show(5)
spark_read_time = datetime.now() - timestamp_start

+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+-----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RatecodeID|store_and_fwd_flag| dropoff_longitude| dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+-----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2016-03-01 00:00:00|  2016-03-01 00:07:55|              1|          2.5|-73.97674560546875| 40.76515197753906|         1|       

In [35]:
timestamp_start = datetime.now()
pd_df = pd.read_csv(csv_path)
pd_df.head(5)
pd_read_time = datetime.now() - timestamp_start

In [36]:
print(f'Spark read time: {spark_read_time}')
print(f'Pandas read time: {pd_read_time}')

Spark read time: 0:00:08.337970
Pandas read time: 0:00:21.229816


In [37]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [38]:
df_schema = df.schema
missing_value_counts = df.agg(*[sum(col(column).isNull().cast('int')).alias(column) for column in df_schema.fieldNames()])
missing_value_counts.show()

+--------+--------------------+---------------------+---------------+-------------+----------------+---------------+----------+------------------+-----------------+----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|RatecodeID|store_and_fwd_flag|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------------+---------------+----------+------------------+-----------------+----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       0|                   0|                    0|              0|            0|               0|              0|         0|                 0|              

In [39]:
selected_columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'tip_amount', 'total_amount']

In [40]:
data = df.select(selected_columns)
data = data.dropna()

In [41]:
data.show(5)

+--------------------+---------------------+---------------+-------------+------------------+------------------+------------------+-----------------+-----------+----------+------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude| dropoff_longitude| dropoff_latitude|fare_amount|tip_amount|total_amount|
+--------------------+---------------------+---------------+-------------+------------------+------------------+------------------+-----------------+-----------+----------+------------+
| 2016-03-01 00:00:00|  2016-03-01 00:07:55|              1|          2.5|-73.97674560546875| 40.76515197753906|-74.00426483154297|40.74612808227539|        9.0|      2.05|       12.35|
| 2016-03-01 00:00:00|  2016-03-01 00:11:06|              1|          2.9|-73.98348236083984| 40.76792526245117|-74.00594329833984| 40.7331657409668|       11.0|      3.05|       15.35|
| 2016-03-01 00:00:00|  2016-03-01 00:31:06|              2|        19